In [1]:
import numpy as np
import tensorflow as tf
#import environment
from tf_agents.environments import py_environment
from tf_agents.environments import tf_py_environment #allows parallel computing for generating experiences
from tf_agents.environments.parallel_py_environment import ParallelPyEnvironment

#agent
from tf_agents.trajectories import StepType


#other used packages
from tf_agents.specs import array_spec
from tf_agents.trajectories import time_step as ts

from typing import Tuple


import collections
import gym

import numpy as np

import statistics
import tensorflow as tf
import tqdm

import function2D as fun

from matplotlib import pyplot as plt
from tensorflow.keras import layers
from typing import Any, List, Sequence, Tuple

2025-06-04 00:00:05.874096: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# !pip install tensorflow==2.15.0
# pip install dm-reverb==0.14.0
# pip install tf-agents==0.19.0

# pip install numpy 

# pip install pygad==3.1.1

# pip3 install -U scikit-learn

# pip install pyswarms==3.10.1


r = np.random.RandomState(0) Creates a NumPy random number generator with seed 0. All random numbers generated via r will now be deterministic and repeatable.
tf.random.set_seed(42) Sets the global seed for TensorFlow's random ops.

Because many ML/RL pipelines use:NumPy for action sampling, data processing, etc.
# TensorFlow for neural networks, gradients, agents.
# To get full reproducibility, you need to set seeds for both libraries.

In [3]:
# Set seed for experiment reproducibility
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# Small epsilon value for stabilizing division operations
eps = np.finfo(np.float32).eps.item()

Define the Environment(from original code Line 135-231) 

 Building a custom environment for reinforcement learning using TF-Agents to optimize a function in continuous space, but with a discrete-like action specification. 

Creating an optimization-style RL environment:

State: a 2D vector (self._state) initialized at x0_reinforce
Action: a vector of shape (2,) where each entry is a discrete int in [0, act_max - act_min]
Goal: update the state using the direction defined by action to maximize a reward computed from the new state

self._action_spec 
minimum=0, maximum=2: So each element of the action vector is an integer in {0, 1, 2}

But why does the action effectively behave like it has only {0, 1}?
normalized_act = (action - self.act_max) / np.sqrt((action - self.act_max)**2 + 1e-7)

action=0     0-1=-1

action=1     1-1=0

action=2     2-1=1
 
 So technically, the action values can be {0, 1, 2}, but when you interpret them through normalization, they behave like directions:

0 → move negative
1 → don't move
2 → move positive

normalized_act ≈ [-1, 0, or 1] per dimension
So action space is discrete with 3 choices per dimension, but the effect on the state is like choosing to move negatively, stay still, or move positively in each dimension.

In [4]:
class Env(py_environment.PyEnvironment):
    def __init__(self, x0_reinforce, act_min = -1, act_max = 1, step_size=0.05, disc_factor = 1.0, sub_episode_length = 30, N=2):
        '''The function to initialize an Env obj. #Initialize environment specs and parameters
        '''
        print("dddd")
        self.state_dim=N ### number of dim
        self.x0_reinforce = x0_reinforce ## Initial state (starting point for each episode)
        self.step_size = step_size  #How much each action changes the state
        self.disc_factor = disc_factor  #Discount factor for rewards (usually 1 in optimization)
        self.sub_episode_length = sub_episode_length #Max steps per episode
        #Specify the requirement for the value of action, (It is a 2d-array for this case)
        #which is an argument of _step(self, action) that is later defined.
        #tf_agents.specs.BoundedArraySpec is a class.
        #_action_spec.check_array( arr ) returns true if arr conforms to the specification stored in _action_spec
        self.act_min =  act_min ## action confined
        self.act_max = act_max #Used to define discrete action range
        
        #Describes the format of actions the agent can take — discrete, vectorized
        #mapping discrete actions to continuous movement directions
        self._action_spec = array_spec.BoundedArraySpec(
                            shape=(self.state_dim,), dtype=np.int32, minimum=0, maximum=self.act_max-self.act_min, name='action') #a_t is an 2darray
     

        # print(self._action_spec)
        #Specify the format requirement for observation (It is a 2d-array for this case), 
        #i.e. the observable part of S_t, and it is stored in self._state
        
        #Describes how the environment represents state (continuous vector)
        self._observation_spec = array_spec.BoundedArraySpec(
                                 shape=(self.state_dim,), dtype=np.float32, name='observation') #default max and min is None
        
        #Current state of the environment
        self._state = np.array(self.x0_reinforce,dtype=np.float32)
        #self.A = mat
        
        #Whether the episode is done
        self._episode_ended = False
        #stop_threshold is a condition for terminating the process for looking for the solution
        #self._stop_threshold = 0.01
        self._step_counter = 0 #Counts steps taken in this episode

    def action_spec(self):
        #return the format requirement for action
        return self._action_spec #Returns the specification of allowed actions, required by TF-Agents.

    def observation_spec(self):
        #return the format requirement for observation
        return self._observation_spec

    def _reset(self):  #Resets the environment to its initial state (at start of each episode).
        self._state = np.array(self.x0_reinforce,dtype=np.float32)  #Resets self._state to x0_reinforce
        self._episode_ended = False
        self._step_counter = 0
        
        #Reward
        initial_r = np.float32(0.0)
        
        #return ts.restart(observation=np.array(self._state, dtype=np.float32))
        return ts.TimeStep(step_type=StepType.FIRST,                            #Returns a TimeStep with:step_type = StepType;FIRSTreward = 0.0;observation = state
                           reward=initial_r, 
                           discount=np.float32(self.disc_factor), 
                           observation=np.array(self._state, dtype=np.float32)
                           )
    
    def set_state(self,new_state):   #set and get state Used to manually manipulate or retrieve the state — useful for testing or external control.
        self._state = new_state 
    
    def get_state(self):
        return self._state
    
    def _step(self, action):
        '''
        The function for the transition from (S_t, A_t) to (R_{t+1}, S_{t+1}).

        Input
        ------
        action: A_t (array or int)

        Output
        -------
        Tuple: (state, reward, done, truncated, info)
        '''
        # If the episode ended in the previous step, reset
        action = np.asarray(action, dtype=np.float32)
        if self._episode_ended:
            return self._reset()  # Should also return (state, info)

        # ---------- Compute S_{t+1} ----------
        normalized_act = (action - self.act_max) / np.sqrt((action - self.act_max)**2 + 1e-7)
        self._state = self._state + normalized_act * self.step_size
        self._step_counter += 1

        # ---------- Compute R_{t+1} ----------
        R = fun.compute_reward(self._state)

        # ---------- Termination condition ----------
        done = self._step_counter >= self.sub_episode_length - 1
        self._episode_ended = done

        # ---------- Return Gym-style step ----------
        return (
            np.array(self._state, dtype=np.float32),  # next state
            float(R),                                 # reward
            1,                                     # done (natural end)
            False,                                    # truncated (optional: implement if needed)
            {}                                        # info
        )

    
    # def _step(self, action):   #Executes one time step: applies action, updates state, computes reward.
    #     '''
    #     The function for the transtion from (S_t, A_t) to (R_{t+1}, S_{t+1}).
    
    #     Input.
    #     --------
    #     self: contain S_t.
    #     action: A_t.
    
    #     Output.
    #     --------
    #     an TimeStep obj, TimeStep(step_type_{t+1}, R_{t+1}, discount_{t+1}, observation S_{t+1})
    #     ''' 
    #     # Suppose that we are at the beginning of time t 
        
    #     ################## --- Determine whether we should end the episode.
    #     if self._episode_ended:  # its time-t value is set at the end of t-1
    #         return self.reset()
    #     # Move on to the following if self._episode_ended=False
        
        
    #     ################# --- Compute S_{t+1} 
    #     #Note that we set the action space as a set of non-negative vectors
    #     #action-act_max converts the set to the desired set of negative vectors.
    #     normalized_act = (action-self.act_max)/np.sqrt((action-self.act_max)**2+0.0000001)    #mapping discrete actions to continuous movement directions
    #     self._state = self._state + normalized_act*self.step_size    
    #     self._step_counter +=1
        
    #     ################# --- Compute R_{t+1}=R(S_t,A_t)
    #     R = fun.compute_reward(self._state)
        
    #     #Set conditions for termination
    #     if self._step_counter>=self.sub_episode_length-1:
    #         self._episode_ended = True  #value for t+1

    #     #Now we are at the end of time t, when self._episode_ended may have changed
    #     if self._episode_ended:
    #         #if self._step_counter>100:
    #         #    reward += np.float32(-100)
    #         #ts.termination(observation,reward,outer_dims=None): Returns a TimeStep obj with step_type set to StepType.LAST.
    #         return ts.termination(np.array(self._state, dtype=np.float32), reward=R)
    #     else:
    #         #ts.transition(observation,reward,discount,outer_dims=None): Returns 
    #         #a TimeStep obj with step_type set to StepType.MID.
    #         return ts.transition(np.array(self._state, dtype=np.float32), reward=R, discount= self.disc_factor)

In [5]:
from tensorflow.keras import layers
from typing import Any, List, Sequence, Tuple
class ActorCritic(tf.keras.Model):
  """Combined actor-critic network."""

  def __init__(
      self,
      num_actions: int,
      num_hidden_units: int):
    """Initialize."""
    super().__init__()

    self.common = layers.Dense(num_hidden_units, activation="relu") #A fully connected (dense) layer shared by both the actor and critic and activation="relu" introduces non-linearity.
    self.actor = layers.Dense(num_actions) #Outputs raw logits for each action (not passed through softmax here).This head is used for the actor, i.e., the policy.
    self.critic = layers.Dense(1) #Outputs a single scalar: the estimated value of the current state. This is the critic.

  def call(self, inputs: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]: #inputs: A batch of states (observations).
    x = self.common(inputs) #Passes the input through the shared dense layer.
    return self.actor(x) #The output x is then: Passed to the actor head → policy logits. Passed to the critic head → scalar value estimate

In [6]:

x0_reinforce = np.array([0.5,-1.0])
EnvTest=Env(x0_reinforce, step_size=10)

dddd


In [7]:
EnvTest._reset()

TimeStep(
{'step_type': array(0, dtype=int32),
 'reward': 0.0,
 'discount': 1.0,
 'observation': array([ 0.5, -1. ], dtype=float32)})

In [8]:
num_actions = EnvTest.action_spec().shape[0]
num_actions

2

In [9]:
num_hidden_units = 128

model = ActorCritic(num_actions, num_hidden_units)

In [10]:
EnvTest._reset().observation

array([ 0.5, -1. ], dtype=float32)

In [11]:
EnvTest._step(EnvTest._reset().observation)

(array([ -9.499999, -11.      ], dtype=float32),
 -0.0024938095426111527,
 1,
 False,
 {})

In [12]:
EnvTest._state

array([ -9.499999, -11.      ], dtype=float32)

In [13]:

# @tf.numpy_function(Tout=[tf.float32, tf.int32, tf.int32])
def env_step(action: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
  """Returns state, reward and done flag given an action."""
  state, reward, done, truncated, info = EnvTest._step(action)
  return (state.astype(np.float32),
          np.array(reward, np.int32),
          np.array(done, np.int32))

In [14]:
def get_expected_return(
    rewards: tf.Tensor,
    standardize: bool = True) -> tf.Tensor:
  """Compute expected returns per timestep."""

  n = tf.shape(rewards)[0]
  returns = tf.TensorArray(dtype=tf.float32, size=n)

  # Start from the end of `rewards` and accumulate reward sums
  # into the `returns` array
  rewards = tf.cast(rewards[::-1], dtype=tf.float32)
  discounted_sum = tf.constant(0.0)
  discounted_sum_shape = discounted_sum.shape
  for i in tf.range(n):
    reward = rewards[i]
    discounted_sum = reward 
    discounted_sum.set_shape(discounted_sum_shape)
    returns = returns.write(i, discounted_sum)
  returns = returns.stack()[::-1]

  if standardize:
    returns = ((returns - tf.math.reduce_mean(returns)) /
               (tf.math.reduce_std(returns) + eps))

  return returns

In [15]:
def run_episode(
    initial_state: tf.Tensor,
    model: tf.keras.Model,
    max_steps: int) -> Tuple[tf.Tensor, tf.Tensor, tf.Tensor]:
  """Runs a single episode to collect training data."""

  action_probs = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
  # values = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
  rewards = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)

  initial_state_shape = initial_state.shape
  state = initial_state

  for t in tf.range(max_steps):
    # Convert state into a batched tensor (batch size = 1)
    state = tf.expand_dims(state, 0)

    # Run the model and to get action probabilities and critic value
    action_logits_t= model(state)

    # Sample next action from the action probability distribution
    
    action = tf.random.categorical(action_logits_t, 1)[0, 0]
    action_probs_t = tf.nn.softmax(action_logits_t)

    # Store critic values
    # values = 0 #values.write(t, tf.squeeze(value))

    # Store log probability of the action chosen
    action_probs = action_probs.write(t, action_probs_t[0, action])

    # Apply action to the environment to get next state and reward
    state, reward, done = env_step(action)
    state.reshape(initial_state_shape)

    # Store reward
    rewards = rewards.write(t, reward)

    if tf.cast(done, tf.bool):
      break

  action_probs = action_probs.stack()
  # values = values.stack()
  rewards = rewards.stack()

  return action_probs, rewards

In [16]:
huber_loss = tf.keras.losses.Huber(reduction=tf.keras.losses.Reduction.SUM)

def compute_loss(
    action_probs: tf.Tensor,
    returns: tf.Tensor) -> tf.Tensor:
  """Computes the combined Actor-Critic loss."""

  advantage = returns

  action_log_probs = tf.math.log(action_probs)
  actor_loss = -tf.math.reduce_sum(action_log_probs * advantage)

  return actor_loss 

In [17]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)


# @tf.function
def train_step( initial_state: tf.Tensor,
    model: tf.keras.Model,
    optimizer: tf.keras.optimizers.Optimizer,
    gamma: float,
    max_steps_per_episode: int) : 
  """Runs a model training step."""
  with tf.GradientTape() as tape:

    # Run the model for one episode to collect training data
    action_probs, rewards = run_episode(
        initial_state, model, max_steps_per_episode)

    # Calculate the expected returns
    returns = get_expected_return(rewards,standardize=True)

    # Convert training data to appropriate TF tensor shapes
    action_probs, returns = [
        tf.expand_dims(x, 1) for x in [action_probs, returns]]

    # Calculate the loss values to update our network
    loss = compute_loss(action_probs, returns)

  # Compute the gradients from the loss
  grads = tape.gradient(loss, model.trainable_variables)

  # Apply the gradients to the model's parameters
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  episode_reward = tf.math.reduce_sum(rewards)

  return episode_reward, loss

In [18]:
%%time

min_episodes_criterion = 100
max_episodes = 10000
max_steps_per_episode = 500

# `CartPole-v1` is considered solved if average reward is >= 475 over 500
# consecutive trials
reward_threshold = 475
running_reward = 0

# The discount factor for future rewards
gamma = 0.99

# Keep the last episodes reward
episodes_reward: collections.deque = collections.deque(maxlen=min_episodes_criterion)

t = tqdm.trange(max_episodes)
for i in t:
    initial_state_info = EnvTest._reset()
    initial_state =initial_state_info.observation
   
    initial_state = tf.constant(initial_state, dtype=tf.float32)
    episode_reward, loss = train_step(
        initial_state, model, optimizer, gamma, max_steps_per_episode)
    
    print("loss", loss)
    episode_reward = int(episode_reward)
    episodes_reward.append(episode_reward)
    running_reward = statistics.mean(episodes_reward)


    t.set_postfix(
        episode_reward=episode_reward, running_reward=running_reward)

    # Show the average episode reward every 10 episodes
    if i % 10 == 0:
      pass # print(f'Episode {i}: average reward: {avg_reward}')

    if running_reward > reward_threshold and i >= min_episodes_criterion:
        break

print(f'\nSolved at episode {i}: average reward: {running_reward:.2f}!')

  0%|          | 5/10000 [00:00<10:21, 16.09it/s, episode_reward=0, running_reward=0]

loss tf.Tensor(0.0, shape=(), dtype=float32)
loss tf.Tensor(0.0, shape=(), dtype=float32)
loss tf.Tensor(0.0, shape=(), dtype=float32)
loss tf.Tensor(0.0, shape=(), dtype=float32)
loss tf.Tensor(0.0, shape=(), dtype=float32)
loss tf.Tensor(0.0, shape=(), dtype=float32)
loss tf.Tensor(0.0, shape=(), dtype=float32)


  0%|          | 13/10000 [00:00<06:04, 27.37it/s, episode_reward=0, running_reward=0]

loss tf.Tensor(0.0, shape=(), dtype=float32)
loss tf.Tensor(0.0, shape=(), dtype=float32)
loss tf.Tensor(0.0, shape=(), dtype=float32)
loss tf.Tensor(0.0, shape=(), dtype=float32)
loss tf.Tensor(0.0, shape=(), dtype=float32)
loss tf.Tensor(0.0, shape=(), dtype=float32)
loss tf.Tensor(0.0, shape=(), dtype=float32)
loss tf.Tensor(0.0, shape=(), dtype=float32)


  0%|          | 21/10000 [00:00<05:13, 31.88it/s, episode_reward=0, running_reward=0]

...

loss tf.Tensor(0.0, shape=(), dtype=float32)
loss tf.Tensor(0.0, shape=(), dtype=float32)
loss tf.Tensor(0.0, shape=(), dtype=float32)
loss tf.Tensor(0.0, shape=(), dtype=float32)
loss tf.Tensor(0.0, shape=(), dtype=float32)
loss tf.Tensor(0.0, shape=(), dtype=float32)
loss tf.Tensor(0.0, shape=(), dtype=float32)
loss tf.Tensor(0.0, shape=(), dtype=float32)


100%|█████████▉| 9993/10000 [05:13<00:00, 31.59it/s, episode_reward=0, running_reward=0]

loss tf.Tensor(0.0, shape=(), dtype=float32)
loss tf.Tensor(0.0, shape=(), dtype=float32)
loss tf.Tensor(0.0, shape=(), dtype=float32)
loss tf.Tensor(0.0, shape=(), dtype=float32)
loss tf.Tensor(0.0, shape=(), dtype=float32)
loss tf.Tensor(0.0, shape=(), dtype=float32)
loss tf.Tensor(0.0, shape=(), dtype=float32)
loss tf.Tensor(0.0, shape=(), dtype=float32)


100%|██████████| 10000/10000 [05:13<00:00, 31.92it/s, episode_reward=0, running_reward=0]


loss tf.Tensor(0.0, shape=(), dtype=float32)
loss tf.Tensor(0.0, shape=(), dtype=float32)
loss tf.Tensor(0.0, shape=(), dtype=float32)
loss tf.Tensor(0.0, shape=(), dtype=float32)

Solved at episode 9999: average reward: 0.00!
CPU times: user 4min 11s, sys: 10.8 s, total: 4min 22s
Wall time: 5min 13s
